In [ ]:
import socket
import subprocess
import requests
import time
import json

In [2]:
with open("../data/o4u_preprocessed_messages_Jun_07_2025.json") as f:
    posts = json.load(f)

posts = posts[:200]

In [3]:
API_URL = "http://localhost:5000/translate"
HEALTH_CHECK_URL = "http://localhost:5000/languages"
USER_LANGUAGE = "ru"
THREAD_COUNTS = [1, 2, 4, 8]
NUM_RUNS = 3

In [4]:
def is_server_ready(timeout: int = 10) -> bool:
    start = time.time()
    while (time.time() - start) < timeout:
        try:
            if requests.get(url=HEALTH_CHECK_URL).status_code == 200:
                return True
        except requests.exceptions.ConnectionError:
            time.sleep(0.5)
    return False


def is_server_down(host: str = "127.0.0.1", port: int = 5000, timeout: int = 20) -> bool:
    start = time.time()
    while (time.time() - start) < timeout:
        try:
            with socket.create_connection(address=(host, port), timeout=1) as sock:
                time.sleep(0.5)
        except (ConnectionRefusedError, TimeoutError):
            return True
    return False

In [5]:
results = {}

for threads in THREAD_COUNTS:
    print(f"=== Testing latency with {threads=} ===")

    start_sh = f"LT_THREADS={threads} docker compose up -d"
    stop_sh = "docker compose down"

    try:
        print("Starting the server")
        print("Waiting...")
        subprocess.run(start_sh, shell=True, check=True)
        if not is_server_ready():
            raise RuntimeError(f"Server did not become ready in time")
        print("Server is ready")

        times = []
        payload = {
            "q": posts,
            "source": "auto",
            "target": USER_LANGUAGE,
            "format": "text"
        }
        print("Starting the benchmark")
        for i in range(NUM_RUNS):
            print(f"Starting run {i+1}/{NUM_RUNS}")
            start = time.time()
            response = requests.post(url=API_URL, json=payload)
            response.raise_for_status()
            delta = time.time() - start
            times.append(delta)
            print(f"Run {i+1}/{NUM_RUNS} completed in {delta:.3f}s")
        average_batch_latency = sum(times) / len(times)
        average_per_post_latency = 1000 * (average_batch_latency / len(posts))
        results[threads] = {
            "average_batch_latency_s": average_batch_latency,
            "average_per_post_latency_ms": average_per_post_latency,
            "batch_size": len(posts)
        }
    except (subprocess.CalledProcessError, requests.exceptions.RequestException, RuntimeError) as e:
        print(e)
        print(f"An error ocurred, {threads=} skipped")
    finally:
        print("Shutting the server down")
        print("Waiting...")
        subprocess.run(stop_sh, shell=True, check=True)
        if not is_server_down(port=5000):
            raise RuntimeError("Server did not shut down")
        print("Server is down")
        print("\n\n")

=== Testing latency with threads=1 ===
Starting the server
Waiting...


 Network machine_translation_default  Creating
 Network machine_translation_default  Created
 Container libretranslate  Creating
 Container libretranslate  Created
 Container libretranslate  Starting
 Container libretranslate  Started


Server is ready
Starting the benchmark
Starting run 1/3
Run 1/3 completed in 114.754s
Starting run 2/3
Run 2/3 completed in 111.426s
Starting run 3/3
Run 3/3 completed in 113.046s
Shutting the server down
Waiting...


 Container libretranslate  Stopping
 Container libretranslate  Stopped
 Container libretranslate  Removing
 Container libretranslate  Removed
 Network machine_translation_default  Removing
 Network machine_translation_default  Removed
 Network machine_translation_default  Creating
 Network machine_translation_default  Created
 Container libretranslate  Creating
 Container libretranslate  Created
 Container libretranslate  Starting


Server is down



=== Testing latency with threads=2 ===
Starting the server
Waiting...


...

 Container libretranslate  Started


Server is ready
Starting the benchmark
Starting run 1/3
Run 1/3 completed in 116.882s
Starting run 2/3
Run 2/3 completed in 113.502s
Starting run 3/3
Run 3/3 completed in 117.546s
Shutting the server down
Waiting...


 Container libretranslate  Stopping
 Container libretranslate  Stopped
 Container libretranslate  Removing
 Container libretranslate  Removed
 Network machine_translation_default  Removing


Server is down





 Network machine_translation_default  Removed


In [6]:
results

{1: {'average_batch_latency_s': 113.07500704129536,
  'average_per_post_latency_ms': 565.3750352064768,
  'batch_size': 200},
 2: {'average_batch_latency_s': 111.87896712621053,
  'average_per_post_latency_ms': 559.3948356310526,
  'batch_size': 200},
 4: {'average_batch_latency_s': 114.56675306955974,
  'average_per_post_latency_ms': 572.8337653477987,
  'batch_size': 200},
 8: {'average_batch_latency_s': 115.97688500086467,
  'average_per_post_latency_ms': 579.8844250043234,
  'batch_size': 200}}

In [ ]:
import sacrebleu


test_data = [
    # --- Short & Direct Announcements ---
    {
        "source_text": "Ultimate Frisbee today at 18:00 by the main IU entrance!",
        "ground_truth_text": "Сегодня в 18:00 у главного входа в Университет Иннополис играем в алтимат фрисби!"
    },
    {
        "source_text": "Poetry evenings ARE BACK!",
        "ground_truth_text": "Поэтические вечера ВОЗВРАЩАЮТСЯ!"
    },
    {
        "source_text": "We are still looking for students to fill various roles for this project!",
        "ground_truth_text": "Мы все еще ищем студентов на различные роли для этого проекта!"
    },
    {
        "source_text": "The deadline for registration was extended!",
        "ground_truth_text": "Срок регистрации был продлен!"
    },
    # --- Event Descriptions ---
    {
        "source_text": "Youth hackathon on creating services for a 'smart building'",
        "ground_truth_text": "Молодежный хакатон по созданию сервисов для 'умного здания'"
    },
    {
        "source_text": "Technical Volunteers needed for DID Summer Forum",
        "ground_truth_text": "Требуются технические волонтеры на Летний форум DID"
    },
    {
        "source_text": "International Fest: International Quest 'Inheritance'",
        "ground_truth_text": "Международный фестиваль: Международный квест 'Наследие'"
    },
    {
        "source_text": "Low Level Programming Club presents a brand-new workshop: Introduction to hobby electronics and IoT with nRF52!",
        "ground_truth_text": "Клуб низкоуровневого программирования представляет новый воркшоп: Введение в хобби-электронику и IoT с nRF52!"
    },
    {
        "source_text": "Guest lecture by Aeroflot CEO advisor, Andrey Polozov-Yablonski on Innovation Management in a Large Company.",
        "ground_truth_text": "Гостевая лекция советника генерального директора Аэрофлота Андрея Полозова-Яблонского на тему Управление инновациями в крупной компании."
    },
    # --- Calls to Action & Invitations ---
    {
        "source_text": "Aikido Innopolis invites students to join their meetings.",
        "ground_truth_text": "Айкидо Иннополис приглашает студентов присоединиться к своим встречам."
    },
    {
        "source_text": "To apply fill the form by 30 November.",
        "ground_truth_text": "Для подачи заявки заполните форму до 30 ноября."
    },
    {
        "source_text": "If you want to work in this project, message AnyaProkhorenko by the end of Friday, November 22.",
        "ground_truth_text": "Если вы хотите работать в этом проекте, напишите AnyaProkhorenko до конца пятницы, 22 ноября."
    },
    {
        "source_text": "Come, support us and enjoy the game!",
        "ground_truth_text": "Приходите поддержать нас и насладиться игрой!"
    },
    # --- Longer, More Complex Descriptions ---
    {
        "source_text": "The course focuses on development of Hyper Casual Games (due to short development cycle and fast marketability response).",
        "ground_truth_text": "Курс сфокусирован на разработке гипер-казуальных игр (ввиду короткого цикла разработки и быстрого отклика рынка)."
    },
    {
        "source_text": "The project is supervised by Admissions Office, who are able to provide you with further information and materials to deliver a presentation at your school.",
        "ground_truth_text": "Проект курируется Приемной комиссией, которая может предоставить вам дополнительную информацию и материалы для проведения презентации в вашей школе."
    },
    {
        "source_text": "The aim of the project is to create a community of entrepreneurs who create products with a national flavor.",
        "ground_truth_text": "Цель проекта - создать сообщество предпринимателей, которые создают продукты с национальным колоритом."
    },
    # --- Questions and Informal Text ---
    {
        "source_text": "Why to stay moody when you can shake your booty?",
        "ground_truth_text": "Зачем грустить, когда можно потанцевать?"
    },
    {
        "source_text": "Experience or creativity - what's more important to win the game?",
        "ground_truth_text": "Опыт или креативность - что важнее для победы в игре?"
    },
    {
        "source_text": "If you can't handle Russian, you'll have to use Google Translate Chrome Extension to be able to navigate through the website.",
        "ground_truth_text": "Если вы не владеете русским языком, вам придется использовать расширение Google Translate для Chrome, чтобы навигироваться по сайту."
    },
    {
        "source_text": "For certain achievements in the ICPC and participation in trainings, you can receive academic and scholarship bonuses.",
        "ground_truth_text": "За определенные достижения в ICPC и участие в тренировках вы можете получить академические и стипендиальные бонусы."
    }
]

source_texts = [item["source_text"] for item in test_data]
payload = {
    "q": source_texts,
    "source": "en",
    "target": "ru"
}
response = requests.post(url=API_URL, json=payload)
machine_translations = response.json().get("translatedText")

human_references = [[item["ground_truth_text"]] for item in test_data]
bleu = sacrebleu.corpus_bleu(machine_translations, human_references)
print(f"Overall Corpus BLEU Score: {bleu.score:.2f}")


Overall Corpus BLEU Score: 53.32

